In [41]:
import sys
sys.path.append("../")
import pandas as pd
import plotly.graph_objects as go
import datetime as dt
from plotting import CandlePlot
from infrastructure.instrument_collection import instrumentCollection as ic



In [42]:
pair = "GBP_JPY"
granularity = "H4"
df = pd.read_pickle(f"./data/{pair}_{granularity}.pkl")
MA_LIST = [10,20,50,100,200]


In [43]:
df.tail()

,time,volume,mid_o,mid_h,mid_l,mid_c,bid_o,bid_h,bid_l,bid_c,ask_o,ask_h,ask_l,ask_c
3995,2025-10-22 01:00:00+00:00,30524,202.752,203.138,202.640,203.044,202.737,203.121,202.624,203.026,202.767,203.155,202.655,203.062
3996,2025-10-22 05:00:00+00:00,40559,203.045,203.271,201.992,202.060,203.028,203.246,201.974,202.046,203.062,203.320,202.009,202.075
3997,2025-10-22 09:00:00+00:00,42032,202.064,202.558,201.924,202.428,202.050,202.543,201.908,202.412,202.078,202.575,201.939,202.445
3998,2025-10-22 13:00:00+00:00,58786,202.426,203.054,202.334,202.926,202.412,203.037,202.319,202.908,202.440,203.071,202.349,202.944
3999,2025-10-22 17:00:00+00:00,31656,202.932,203.052,202.532,202.964,202.916,203.036,202.515,202.926,202.949,203.069,202.550,203.002


In [44]:
# turn dates type to string to avoid having weekend gaps on chart
# this is a workaround, better solution would be to use rangebreaks in plotly
df['sTime']=[dt.datetime.strftime(x,"s%y-%m-%d %H:%M") for x in df.time] 

In [45]:

df_plot = df.iloc[:200] # plot only first 200 rows for clarity

In [46]:
fig = go.Figure()
fig.add_trace(go.Candlestick(
    x=df_plot.sTime,
    open=df_plot.mid_o,
     high=df_plot.mid_h,
     low=df_plot.mid_l,
     close=df_plot.mid_c,
    line=dict(width=1), opacity=1,
    increasing_fillcolor="#24A06B",
    decreasing_fillcolor="#CC2E3C",
    increasing_line_color="#2EC886",
    decreasing_line_color="#FF3A4C",
))
fig.update_yaxes(
    gridcolor="#1f292f"
)
fig.update_xaxes(
    gridcolor="#1f292f",
    rangeslider=dict(visible=False),
    nticks=5
)
fig.update_layout(
    width=900,
    height=400,
    margin=dict(l=10,r=10,b=10,t=10),
    paper_bgcolor="#2c303c",
    plot_bgcolor="#2c303c",
    font=dict(size=8, color="#e1e1e1"),
)
fig.show()

In [47]:
df['time']

0      2023-03-29 05:00:00+00:00
1      2023-03-29 09:00:00+00:00
2      2023-03-29 13:00:00+00:00
3      2023-03-29 17:00:00+00:00
4      2023-03-29 21:00:00+00:00
                  ...           
3995   2025-10-22 01:00:00+00:00
3996   2025-10-22 05:00:00+00:00
3997   2025-10-22 09:00:00+00:00
3998   2025-10-22 13:00:00+00:00
3999   2025-10-22 17:00:00+00:00
Name: time, Length: 4000, dtype: datetime64[ns, tzutc()]

In [48]:
df_ma = df[['time', 'mid_o', 'mid_h', 'mid_l', 'mid_c']].copy()
df_ma.head()

,time,mid_o,mid_h,mid_l,mid_c
0,2023-03-29 05:00:00+00:00,162.386,162.770,162.265,162.681
1,2023-03-29 09:00:00+00:00,162.678,163.486,162.324,163.426
2,2023-03-29 13:00:00+00:00,163.426,163.464,162.894,163.278
3,2023-03-29 17:00:00+00:00,163.278,163.688,163.198,163.656
4,2023-03-29 21:00:00+00:00,163.557,163.557,162.980,163.220


In [49]:
for ma in MA_LIST:
    df_ma[f'MA_{ma}'] = df_ma.mid_c.rolling(window=ma).mean()


In [50]:
df_ma.dropna(inplace=True) # remove rows with NaN values
df_ma.reset_index(drop=True, inplace=True) # reset index after dropping rows
df_ma.head()

,time,mid_o,mid_h,mid_l,mid_c,MA_10,MA_20,MA_50,MA_100,MA_200
0,2023-05-15 09:00:00+00:00,169.987,170.428,169.812,169.972,169.1272,169.25100,169.62748,168.98620,167.118345
1,2023-05-15 13:00:00+00:00,169.968,170.586,169.950,170.555,169.3427,169.24185,169.64074,169.02275,167.157715
2,2023-05-15 17:00:00+00:00,170.561,170.626,170.388,170.515,169.5392,169.25060,169.66648,169.06344,167.193160
3,2023-05-15 21:00:00+00:00,170.475,170.518,170.283,170.324,169.6816,169.28965,169.67404,169.10786,167.228390
4,2023-05-16 01:00:00+00:00,170.325,170.444,170.158,170.340,169.8353,169.32275,169.69444,169.15232,167.261810


In [51]:





df_plot = df_ma.iloc[:500] # plot only first 100 rows for clarity
cp = CandlePlot(df_plot)
# cp.fig.add_trace(go.Scatter(
#     x=cp.df_plot.sTime,
#     y=cp.df_plot.MA_10,
#     line=dict(color='#FFD700', width=2),
#     name='MA 10',
#     line_shape='spline',
# ))
traces = [f"MA_{ma}" for ma in MA_LIST]
cp.show_plot(line_traces=traces)

In [52]:
MA_SHORT = "MA_10"
MA_LONG = "MA_20"
BUY = 1
SELL = -1
NONE = 0
df_an = df_ma[['time', 'mid_o', 'mid_h', 'mid_l', 'mid_c', MA_SHORT, MA_LONG]].copy()
df_an.head()

,time,mid_o,mid_h,mid_l,mid_c,MA_10,MA_20
0,2023-05-15 09:00:00+00:00,169.987,170.428,169.812,169.972,169.1272,169.25100
1,2023-05-15 13:00:00+00:00,169.968,170.586,169.950,170.555,169.3427,169.24185
2,2023-05-15 17:00:00+00:00,170.561,170.626,170.388,170.515,169.5392,169.25060
3,2023-05-15 21:00:00+00:00,170.475,170.518,170.283,170.324,169.6816,169.28965
4,2023-05-16 01:00:00+00:00,170.325,170.444,170.158,170.340,169.8353,169.32275


In [53]:
df_an['DELTA'] = df_an.MA_10 - df_an.MA_20
df_an['DELTA_PREV'] = df_an.DELTA.shift(1)
df_an

,time,mid_o,mid_h,mid_l,mid_c,MA_10,MA_20,DELTA,DELTA_PREV
0,2023-05-15 09:00:00+00:00,169.987,170.428,169.812,169.972,169.1272,169.25100,-0.12380,NaN
1,2023-05-15 13:00:00+00:00,169.968,170.586,169.950,170.555,169.3427,169.24185,0.10085,-0.12380
2,2023-05-15 17:00:00+00:00,170.561,170.626,170.388,170.515,169.5392,169.25060,0.28860,0.10085
3,2023-05-15 21:00:00+00:00,170.475,170.518,170.283,170.324,169.6816,169.28965,0.39195,0.28860
4,2023-05-16 01:00:00+00:00,170.325,170.444,170.158,170.340,169.8353,169.32275,0.51255,0.39195
...,...,...,...,...,...,...,...,...,...
3796,2025-10-22 01:00:00+00:00,202.752,203.138,202.640,203.044,202.7032,202.36045,0.34275,0.31435
3797,2025-10-22 05:00:00+00:00,203.045,203.271,201.992,202.060,202.6940,202.36250,0.33150,0.34275
3798,2025-10-22 09:00:00+00:00,202.064,202.558,201.924,202.428,202.7273,202.39700,0.33030,0.33150
3799,2025-10-22 13:00:00+00:00,202.426,203.054,202.334,202.926,202.8103,202.48965,0.32065,0.33030


In [54]:
def isTrade(row):
    if row.DELTA > 0 and row.DELTA_PREV <= 0:
        return BUY
    elif row.DELTA < 0 and row.DELTA_PREV >= 0:
        return SELL
    else:
        return NONE

In [55]:
ic.LoadInstruments("../exploration/data")
ins_data = ic.instruments_dict[pair]

In [56]:
df_an['Signal'] = df_an.apply(isTrade, axis=1).copy()
# create a dataframe with only the trade signals
df_signals = df_an[df_an.Signal != NONE].copy()
df_signals

,time,mid_o,mid_h,mid_l,mid_c,MA_10,MA_20,DELTA,DELTA_PREV,Signal
1,2023-05-15 13:00:00+00:00,169.968,170.586,169.950,170.555,169.3427,169.24185,0.10085,-0.12380,1
72,2023-05-31 09:00:00+00:00,172.870,173.171,172.558,172.852,173.3384,173.34830,-0.00990,0.08175,-1
81,2023-06-01 21:00:00+00:00,173.928,174.070,173.752,174.045,173.5123,173.45825,0.05405,-0.03450,1
95,2023-06-06 05:00:00+00:00,173.622,173.674,172.874,172.918,173.7489,173.83360,-0.08470,0.02860,-1
106,2023-06-08 01:00:00+00:00,174.202,174.380,173.912,174.017,173.7499,173.70295,0.04695,-0.03335,1
...,...,...,...,...,...,...,...,...,...,...
3723,2025-10-05 21:00:00+00:00,200.827,201.315,200.238,201.208,198.5740,198.51150,0.06250,-0.11570,1
3750,2025-10-10 09:00:00+00:00,203.143,203.212,202.496,202.590,203.7505,203.84230,-0.09180,0.11530,-1
3776,2025-10-16 17:00:00+00:00,201.940,202.248,201.758,202.114,202.4509,202.42075,0.03015,-0.03335,1
3782,2025-10-17 17:00:00+00:00,201.808,202.269,201.718,202.260,202.1094,202.16575,-0.05635,0.00955,-1


In [57]:
 # calculate difference between mid_c and previous mid_c
df_signals['DIFF'] = df_signals.mid_c.diff().shift(-1)
df_signals.fillna(0, inplace=True)
df_signals

,time,mid_o,mid_h,mid_l,mid_c,MA_10,MA_20,DELTA,DELTA_PREV,Signal,DIFF
1,2023-05-15 13:00:00+00:00,169.968,170.586,169.950,170.555,169.3427,169.24185,0.10085,-0.12380,1,2.297
72,2023-05-31 09:00:00+00:00,172.870,173.171,172.558,172.852,173.3384,173.34830,-0.00990,0.08175,-1,1.193
81,2023-06-01 21:00:00+00:00,173.928,174.070,173.752,174.045,173.5123,173.45825,0.05405,-0.03450,1,-1.127
95,2023-06-06 05:00:00+00:00,173.622,173.674,172.874,172.918,173.7489,173.83360,-0.08470,0.02860,-1,1.099
106,2023-06-08 01:00:00+00:00,174.202,174.380,173.912,174.017,173.7499,173.70295,0.04695,-0.03335,1,6.971
...,...,...,...,...,...,...,...,...,...,...,...
3723,2025-10-05 21:00:00+00:00,200.827,201.315,200.238,201.208,198.5740,198.51150,0.06250,-0.11570,1,1.382
3750,2025-10-10 09:00:00+00:00,203.143,203.212,202.496,202.590,203.7505,203.84230,-0.09180,0.11530,-1,-0.476
3776,2025-10-16 17:00:00+00:00,201.940,202.248,201.758,202.114,202.4509,202.42075,0.03015,-0.03335,1,0.146
3782,2025-10-17 17:00:00+00:00,201.808,202.269,201.718,202.260,202.1094,202.16575,-0.05635,0.00955,-1,0.064


In [58]:
df_signals['Gain'] = df_signals['DIFF'] / ins_data.pip_location
df_signals['Gain'] = df_signals['Gain'] * df_signals['Signal']
df_signals

,time,mid_o,mid_h,mid_l,mid_c,MA_10,MA_20,DELTA,DELTA_PREV,Signal,DIFF,Gain
1,2023-05-15 13:00:00+00:00,169.968,170.586,169.950,170.555,169.3427,169.24185,0.10085,-0.12380,1,2.297,229.7
72,2023-05-31 09:00:00+00:00,172.870,173.171,172.558,172.852,173.3384,173.34830,-0.00990,0.08175,-1,1.193,-119.3
81,2023-06-01 21:00:00+00:00,173.928,174.070,173.752,174.045,173.5123,173.45825,0.05405,-0.03450,1,-1.127,-112.7
95,2023-06-06 05:00:00+00:00,173.622,173.674,172.874,172.918,173.7489,173.83360,-0.08470,0.02860,-1,1.099,-109.9
106,2023-06-08 01:00:00+00:00,174.202,174.380,173.912,174.017,173.7499,173.70295,0.04695,-0.03335,1,6.971,697.1
...,...,...,...,...,...,...,...,...,...,...,...,...
3723,2025-10-05 21:00:00+00:00,200.827,201.315,200.238,201.208,198.5740,198.51150,0.06250,-0.11570,1,1.382,138.2
3750,2025-10-10 09:00:00+00:00,203.143,203.212,202.496,202.590,203.7505,203.84230,-0.09180,0.11530,-1,-0.476,47.6
3776,2025-10-16 17:00:00+00:00,201.940,202.248,201.758,202.114,202.4509,202.42075,0.03015,-0.03335,1,0.146,14.6
3782,2025-10-17 17:00:00+00:00,201.808,202.269,201.718,202.260,202.1094,202.16575,-0.05635,0.00955,-1,0.064,-6.4


In [59]:
df_signals['Gain_Cum'] = df_signals.Gain.cumsum()
df_signals.head()

,time,mid_o,mid_h,mid_l,mid_c,MA_10,MA_20,DELTA,DELTA_PREV,Signal,DIFF,Gain,Gain_Cum
1,2023-05-15 13:00:00+00:00,169.968,170.586,169.950,170.555,169.3427,169.24185,0.10085,-0.12380,1,2.297,229.7,229.7
72,2023-05-31 09:00:00+00:00,172.870,173.171,172.558,172.852,173.3384,173.34830,-0.00990,0.08175,-1,1.193,-119.3,110.4
81,2023-06-01 21:00:00+00:00,173.928,174.070,173.752,174.045,173.5123,173.45825,0.05405,-0.03450,1,-1.127,-112.7,-2.3
95,2023-06-06 05:00:00+00:00,173.622,173.674,172.874,172.918,173.7489,173.83360,-0.08470,0.02860,-1,1.099,-109.9,-112.2
106,2023-06-08 01:00:00+00:00,174.202,174.380,173.912,174.017,173.7499,173.70295,0.04695,-0.03335,1,6.971,697.1,584.9


In [60]:
cp = CandlePlot(df_signals, candles=False)
cp.show_plot(line_traces=['Gain_Cum'])